<a href="https://colab.research.google.com/github/maniezhilan/Trading-Algos/blob/master/Research%2C_Backtest%2C_Trade_From_Jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Alpaca Google Colab Instant Development Environment! 

Alpaca offers free live and historical market data for stocks and cryptocurrencies, and also offers free paper and live trading environments. These services are accessed by API and you can use them for a variety of things, like research and backtesting using market data, paper trading crypto, etc. 

This notebook allows you to quickly get started with Alpaca and start exploring what you can build. 

You can use this notebook to research new ideas, backtest them and even place trades.


# Table of Contents
1. [Introduction](#first-bullet)
2. [Accessing Market Data](#second-bullet)
3. [Backtesting](#third-bullet)
4. [Paper Trading](#fourth-bullet)

#Introduction <a class="anchor" id="first-bullet"></a>

To follow along, you'll need to use your Alpaca API paper keys.

If you don't have keys, sign up here:
[Free Stock and Crypto Data with $0 Commission Trading](https://app.alpaca.markets/signup)

You can find your keys on your [dashboard](https://app.alpaca.markets/paper/dashboard/overview). If you're having trouble finding your keys, follow [this tutorial](https://alpaca.markets/learn/connect-to-alpaca-api/). 

In [ ]:
API_KEY = '<Your API Key>'
SECRET_KEY = '<Your API Secret Key>'

Now we can get started! We will need to import `alpaca_trade_api`. 

If we haven't installed it yet, we can do so by running `pip install alpaca_trade_api`.

Then let's define an instance of the REST API interface and provide it our API keys.

In [ ]:
%%capture
!pip install alpaca_trade_api
!pip install backtrader
!pip install matplotlib==3.2.2
!pip install plotly

In [ ]:
from alpaca_trade_api.rest import REST, TimeFrame
from alpaca_trade_api.stream import Stream

rest_api = REST(API_KEY, SECRET_KEY, 'https://paper-api.alpaca.markets')

#Accessing Market Data  <a class="anchor" id="second-bullet"></a>

The REST API allows you to access historical market data, account data, and place trades.

Here are some of the types of the market data Alpaca offers:
*   Bar data is available in custom timeframes (1 day, 30 mins, 5 mins, etc)
*   Quote and Trade data is also available

Let's take at daily bar data for SPY between Jan 1st 2021 and Jan 30th 2021.

In [ ]:
# retrieve daily bar data for SPY in a dataframe 
spy_bars = rest_api.get_bars('SPY', TimeFrame.Day, '2021-01-01', '2021-03-30').df
spy_bars

# quote and trade data also available for equities
# spy_quotes = rest_api.get_quotes('SPY', '2021-01-01', '2021-01-05').df
# spy_trades = rest_api.get_trades('SPY', '2021-01-01', '2021-01-05').df

HTTPError: ignored

Historical market data is also available for cryptocurrencies. 

Currently, alpaca supports `Bitcoin (BTCUSD)`, `Ethereum (ETHUSD)`, `Litecoin (LTCUSD)`, and `Bitcoin Cash (BCHUSD)`

Let's take a look at quote data for Bitcoin between Jan 1st 2020 and Jan 1st 2021.

In [ ]:
# retrieve quote data for Bitcoin in a dataframe
bitcoin_quotes = rest_api.get_crypto_quotes('BTCUSD', '2021-01-01', '2021-01-01').df
bitcoin_quotes

# bar and trade data also available for crypto
# bitcoin_bars = rest_api.get_crypto_bars('BTCUSD', TimeFrame.Day, '2020-01-01', '2021-01-05').df
# bitcoin_trades = rest_api.get_crypto_trades('BTCUSD', '2021-01-01', '2021-01-05').df

Using Jupyter notebooks allow you to easily visualize data and test ideas. 

For example, we can use plotly to make beautiful candle charts of SPY's daily bar data. 

We can also calculate the 13 day simple moving average (SMA) for SPY over that period using pandas operations, and add it onto our chart.

In [ ]:
# plotly imports
import plotly.graph_objects as go
import plotly.express as px

# SPY bar data candlestick plot
candlestick_fig = go.Figure(data=[go.Candlestick(x=spy_bars.index,
                open=spy_bars['open'],
                high=spy_bars['high'],
                low=spy_bars['low'],
                close=spy_bars['close'])])

# calculating 13 day SMA using pandas rolling mean
sma = spy_bars['close'].rolling(13).mean().dropna()

# creating a line plot for our sma
sma_fig = px.line(x=sma.index, y=sma)

# adding both plots onto one chart
fig = go.Figure(data=candlestick_fig.data + sma_fig.data)

# displaying our chart
fig.show()

#Backtesting  <a class="anchor" id="third-bullet"></a>

We can use backtrader along with Alpaca’s Market Data API to backtest a strategy’s performance with historical data. Backtrader allows you to focus on writing reusable trading strategies, indicators and analyzers instead of having to spend time building infrastructure. 

Below we wrote a function that allows you to easily backtest a strategy written in backtrader. The function takes in as a parameter the strategy you wish to backtest, the symbols the strategy trades on, start and end dates of the backtest, the timeframe of the strategy, and the initial cash of the backtest. 


In [ ]:
import backtrader as bt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

def run_backtest(strategy, symbols, start, end, timeframe=TimeFrame.Day, cash=10000):
 """params:
       strategy: the strategy you wish to backtest, an instance of backtrader.Strategy
       symbols: the symbol (str) or list of symbols List[str] you wish to backtest on
       start: start date of backtest in format 'YYYY-MM-DD'
       end: end date of backtest in format: 'YYYY-MM-DD'
       timeframe: the timeframe the strategy trades on (size of bars) -
                   1 min: TimeFrame.Minute, 1 day: TimeFrame.Day, 5 min: TimeFrame(5, TimeFrameUnit.Minute)
       cash: the starting cash of backtest
 """

 # initialize backtrader broker
 cerebro = bt.Cerebro(stdstats=True)
 cerebro.broker.setcash(cash)

 # add strategy
 cerebro.addstrategy(strategy)

 # add analytics
 # cerebro.addobserver(bt.observers.Value)
 # cerebro.addobserver(bt.observers.BuySell)
 cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
  # historical data request
 if type(symbols) == str:
   symbol = symbols
   alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
   data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
   cerebro.adddata(data)
 elif type(symbols) == list or type(symbols) == set:
   for symbol in symbols:
     alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
     data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
     cerebro.adddata(data)


 # run
 initial_portfolio_value = cerebro.broker.getvalue()
 print(f'Starting Portfolio Value: {initial_portfolio_value}')
 results = cerebro.run()
 final_portfolio_value = cerebro.broker.getvalue()
 print(f'Final Portfolio Value: {final_portfolio_value} ---> Return: {(final_portfolio_value/initial_portfolio_value - 1)*100}%')

 strat = results[0]
 print('Sharpe Ratio:', strat.analyzers.mysharpe.get_analysis()['sharperatio'])
 cerebro.plot(iplot= False)


## Backtesting Example 1: Simple Moving Average Crossover

Below is an example of a simple backtrader strategy. It is a simple moving average crossover strategy which trades when the 13 day SMA crosses over the 25 day SMA, and sells when it crosses under.  In this example, a crossover indicator is created which is 1 when the fast SMA crosses over the slow SMA, -1 when the fast crosses below the slow, and 0 if there is no crossover.

Take note of the structure of the strategy. We can define parameters for the strategy in the `params` dictionary. Any indicators can be defined in `__init__`. You can find a full list of indicators in the backtrader documentation. The `next()` method is called each iteration of the backtest. 


At the end you can run `run_backtest(SmaCross, 'AAPL', '2019-01-01', '2021-11-01', TimeFrame.Day, 10000)`, which will compute the backtest and output the results.



In [ ]:
class SmaCross(bt.Strategy):
  
  # list of parameters which are configurable for the strategy
    params = dict(
        pfast=13,  # period for the fast moving average
        pslow=25   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
  
    def next(self):
        if not self.position and self.crossover > 0:  # not in the market
            self.buy()
           
        elif self.position and self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position


run_backtest(SmaCross, 'AAPL', '2019-01-01', '2021-11-01', TimeFrame.Day, 10000)

## Backtesting Example 2: All Weather Portfolio

Another class of strategies are rebalancing strategies. These strategies attempt to match a desired portfolio. Some rebalancing strategies change the securities in their portfolio based on market conditions, others maintain a steady allocation through all conditions. 

A famous rebalancing strategy is called the [All Weather Portfolio](https://www.optimizedportfolio.com/all-weather-portfolio/) by Ray Dalio. It attempts to produce steady returns through all market conditions by investing in a variety of sectors. 

We'll create the portfolio by allocating to these industry ETFs.
30% Vanguard Total Stock Market ETF (VTI)
* 40% iShares 20+ Year Treasury ETF (TLT)
* 15% iShares 7 – 10 Year Treasury ETF (IEF)
* 7.5% SPDR Gold Shares ETF (GLD)
* 7.5% PowerShares DB Commodity Index Tracking Fund (DBC)


In [ ]:
class AllWeatherStrategy(bt.Strategy):

   def __init__(self):
       # the last year we rebalanced (initialized to -1)
       self.year_last_rebalanced = -1 
       self.weights = { "VTI" : 0.30 , "TLT" : 0.40, "IEF": 0.15, "GLD" : 0.075, "DBC" : 0.075 }

   def next(self):
       # if we’ve already rebalanced this year
       if self.datetime.date().year == self.year_last_rebalanced:
           return
       # update year last balanced
       self.year_last_rebalanced = self.datetime.date().year
       # enumerate through each security
       for i,d in enumerate(self.datas):
           # rebalance portfolio with desired target percents
           symbol = d._name
           self.order_target_percent(d, target=self.weights[symbol])

run_backtest(AllWeatherStrategy, ["VTI", "TLT", "IEF", "GLD", "DBC"] , '2015-01-01', '2021-11-01', TimeFrame.Day, 10000)

# Paper Trading  <a class="anchor" id="fourth-bullet"></a>

We can use Alpaca’s trade API to place paper trades on our Alpaca account. Paper trading is free and doesn’t require you to fund your account. Let’s place trades so that we can create the All Weather Portfolio in our paper account.





In [ ]:
import random 

percent_allocations = {'VTI': 0.30, 'TLT': 0.40, 'IEF': 0.15, 'GLD': 0.075, 'DBC': 0.075}


# get total account value
account_equity = float(rest_api.get_account().equity)


# how many dollars we want to allocate to each symbol
dollar_value_allocations = {symbol: percent * account_equity for symbol, percent in percent_allocations.items()}

# liquidate all existing positions before rebalancing
rest_api.close_all_positions()


# Rebalance portfolio
for symbol, dollars_alloc in dollar_value_allocations.items():
     
    # market price of current ETF
    market_price = rest_api.get_latest_bar(symbol).close
    
    # how many shares we want
    target_holdings = dollars_alloc // market_price
     
    # how many shares we have to buy to match target
    order_quantity = target_holdings
     
    # submit market order for this ETF
    print(f"Submitting market order for {order_quantity} shares of {symbol}")
    rest_api.submit_order(symbol, order_quantity, 'buy', 'market', client_order_id=f'colab_{random.randrange(10000000)}')